In [ ]:
import warnings

warnings.filterwarnings("ignore")

import os
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import pandas as pd
import numpy as np
import random
import sc_toolbox

import rpy2.rinterface_lib.callbacks
import anndata2ri
import logging

from rpy2.robjects import pandas2ri
from rpy2.robjects import r

sc.settings.verbosity = 0
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

# Initialize random seed
import random
random.seed(111)

# set a working directory
# wdir = "/ceph/project/tendonhca/akurjan/analysis/"
wdir = "/mnt/da8aa2c4-0136-465b-87a2-d12a59afec55/akurjan/analysis/notebooks"
os.chdir( wdir )

# folder structures
SCVI_FOLDERNAME = "foetal/results/scVI/"
RESULTS_FOLDERNAME = "foetal/results/DGE/"
FIGURES_FOLDERNAME = "foetal/figures/DGE/"

if not os.path.exists(RESULTS_FOLDERNAME):
    os.makedirs(RESULTS_FOLDERNAME)
if not os.path.exists(FIGURES_FOLDERNAME):
    os.makedirs(FIGURES_FOLDERNAME)

# Set folder for saving figures into
sc.settings.figdir = FIGURES_FOLDERNAME

# Print date and time:
import datetime
e = datetime.datetime.now()
print ("Current date and time = %s" % e)

# Set other settings
sc.settings.verbosity = 3 # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
sc.set_figure_params(dpi=150, fontsize=10, dpi_save=600)

In [ ]:
%%R
#BiocManager::install("edgeR")
library(edgeR)
#library(MAST)

# Pseudobulk Data Preparation

In [ ]:
adata = sc.read_h5ad(os.path.join(SCVI_FOLDERNAME, 'dev_scANVI.h5ad'))
adata

In [ ]:
np.max(adata.layers['counts'])

In [ ]:
np.max(adata.X)

In [ ]:
adata.X = adata.layers['log1p_norm'].copy()
np.max(adata.X)

In [ ]:
print(len(adata[adata.obs["age"] == "12w"].obs["sampletype"].cat.categories))
print(len(adata[adata.obs["age"] == "17w"].obs["sampletype"].cat.categories))
print(len(adata[adata.obs["age"] == "20w"].obs["sampletype"].cat.categories))

In [ ]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
adata

In [ ]:
adata.obs["bulksample"] = [
    f"{rep}_{l}" for rep, l in zip(adata.obs["sampletype"], adata.obs["age"])
]

In [ ]:
adata.obs["bulksample"] 

In [ ]:
adata.obs["C_scANVI"] = [ct.replace(" ", "_") for ct in adata.obs["C_scANVI"]]
adata.obs["C_scANVI"] = [ct.replace("+", "") for ct in adata.obs["C_scANVI"]]
adata.obs["C_scANVI"]

In [ ]:
adata.obs["sampletype"] = adata.obs["sampletype"].astype("category")
adata.obs["age"] = adata.obs["age"].astype("category")
adata.obs["bulksample"] = adata.obs["bulksample"].astype("category")
adata.obs["C_scANVI"] = adata.obs["C_scANVI"].astype("category")

In [ ]:
adata.obs["C_scANVI"].value_counts()

In [ ]:
NUM_OF_CELL_PER_DONOR = 30 # to filter out donors with less than that amount of cells


def aggregate_and_filter(
    adata,
    cell_identity,
    donor_key="bulksample",
    condition_key="age",
    cell_identity_key="C_scANVI",
    obs_to_keep=[],  # which additional metadata to keep, e.g. gender, age, etc.
    replicates_per_patient=1,
):
    # subset adata to the given cell identity
    adata_cell_pop = adata[adata.obs[cell_identity_key] == cell_identity].copy()
    # check which donors to keep according to the number of cells specified with NUM_OF_CELL_PER_DONOR
    size_by_donor = adata_cell_pop.obs.groupby([donor_key]).size()
    donors_to_drop = [
        donor
        for donor in size_by_donor.index
        if size_by_donor[donor] <= NUM_OF_CELL_PER_DONOR
    ]
    if len(donors_to_drop) > 0:
        print("Dropping the following samples:")
        print(donors_to_drop)
    df = pd.DataFrame(columns=[*adata_cell_pop.var_names, *obs_to_keep])

    adata_cell_pop.obs[donor_key] = adata_cell_pop.obs[donor_key].astype("category")
    donors = adata_cell_pop.obs[donor_key].cat.categories
    for i, donor in enumerate(donors):
    #for i, donor in enumerate(donors := adata_cell_pop.obs[donor_key].cat.categories): - can't use walrus operator here
        print(f"\tProcessing donor {i+1} out of {len(donors)}...", end="\r")
        if donor not in donors_to_drop:
            adata_donor = adata_cell_pop[adata_cell_pop.obs[donor_key] == donor]
            # create replicates for each donor
            indices = list(adata_donor.obs_names)
            random.shuffle(indices)
            indices = np.array_split(np.array(indices), replicates_per_patient)
            for i, rep_idx in enumerate(indices):
                adata_replicate = adata_donor[rep_idx]
                # specify how to aggregate: sum gene expression for each gene for each donor and also keep the condition information
                agg_dict = {gene: "sum" for gene in adata_replicate.var_names}
                for obs in obs_to_keep:
                    agg_dict[obs] = "first"
                # create a df with all genes, donor and condition info
                df_donor = pd.DataFrame(adata_replicate.X.A)
                df_donor.index = adata_replicate.obs_names
                df_donor.columns = adata_replicate.var_names
                df_donor = df_donor.join(adata_replicate.obs[obs_to_keep])
                # aggregate
                df_donor = df_donor.groupby(donor_key).agg(agg_dict)
                df_donor[donor_key] = donor
                df.loc[f"donor_{donor}_{i}"] = df_donor.loc[donor]
    print("\n")
    # create AnnData object from the df
    adata_cell_pop = sc.AnnData(
        df[adata_cell_pop.var_names], obs=df.drop(columns=adata_cell_pop.var_names)
    )
    return adata_cell_pop

In [ ]:
obs_to_keep = ["age", "C_scANVI", "bulksample", "sex", "type", "sample", "libbatch", "sampletype"]

In [ ]:
adata.X = adata.layers["counts"].copy()

In [ ]:
# process first cell type separately...
cell_type = adata.obs["C_scANVI"].cat.categories[0]
print(
    f'Processing {cell_type} (1 out of {len(adata.obs["C_scANVI"].cat.categories)})...'
)

adata_pb = aggregate_and_filter(adata, cell_type, obs_to_keep=obs_to_keep)
for i, cell_type in enumerate(adata.obs["C_scANVI"].cat.categories[1:]):
    print(
        f'Processing {cell_type} ({i+2} out of {len(adata.obs["C_scANVI"].cat.categories)})...'
    )
    adata_cell_type = aggregate_and_filter(adata, cell_type, obs_to_keep=obs_to_keep)
    adata_pb = adata_pb.concatenate(adata_cell_type)

In [ ]:
adata_pb.layers['counts'] = adata_pb.X.copy()

In [ ]:
sc.pp.normalize_total(adata_pb, target_sum=1e6)
sc.pp.log1p(adata_pb)
sc.pp.pca(adata_pb)

In [ ]:
adata_pb.obs["lib_size"] = np.sum(adata_pb.layers["counts"], axis=1)
adata_pb.obs["log_lib_size"] = np.log(adata_pb.obs["lib_size"])
sc.pl.pca(adata_pb, color=adata_pb.obs, ncols=1, size=300, save='dev_dge_pca.svg')

In [ ]:
adata_pb.obs_names = [
    name.split("_")[1] + "_" + name.split("_")[2] + "_" + name.split("_")[3] for name in adata_pb.obs_names
]

In [ ]:
adata_pb.obs

In [ ]:
adata_pb.var

In [ ]:
%%R
fit_model <- function(adata_){
    # create an edgeR object with counts and grouping factor (condition)
    y <- DGEList(assay(adata_, "X"), group = colData(adata_)$age)
    # filter out genes with low counts
    print("Dimensions before subsetting:")
    print(dim(y))
    print("")
    keep <- filterByExpr(y)
    y <- y[keep, , keep.lib.sizes=FALSE]
    print("Dimensions after subsetting:")
    print(dim(y))
    print("")
    # normalize
    y <- calcNormFactors(y)
    # create a vector that is concatentation of condition and cell type that we will later use with contrasts
    group <- paste0(colData(adata_)$age, ".", colData(adata_)$C_scANVI)
    replicate <- colData(adata_)$sampletype
    #batch <- colData(adata_)$libbatch
    # create a design matrix: here we have multiple donors so also consider that in the design matrix
    design <- model.matrix(~ 0 + group + replicate)
    # estimate dispersion
    y <- estimateDisp(y, design = design)
    # fit the model
    fit <- glmQLFit(y, design)
    return(list("fit"=fit, "design"=design, "y"=y))
}

In [ ]:
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

#adata_mono_r = robjects.conversion.py2rpy(adata_mono)

robjects.globalenv['adata_pb'] = adata_pb
robjects.r('set.seed(123)')
robjects.r('library("edgeR")')
robjects.r('y <- DGEList(assay(adata_pb, "X"), group = colData(adata_pb)$age)')
robjects.r('print("Dimensions before subsetting:")')
robjects.r('print(dim(y))')
robjects.r('print("")')
robjects.r('keep <- filterByExpr(y, min.count = 5, min.total.count = 15)') 
# min.count is the minimum count threshold for a gene to be considered expressed in a sample.
# min.total.count is the minimum total count threshold for a gene to be considered expressed across all samples.
robjects.r('y <- y[keep, , keep.lib.sizes=FALSE]')
robjects.r('print("Dimensions after subsetting:")')
robjects.r('print(dim(y))')
robjects.r('print("")')
robjects.r('y <- calcNormFactors(y)')
robjects.r('group <- paste0(colData(adata_pb)$age, ".", colData(adata_pb)$C_scANVI)')
robjects.r('replicate <- colData(adata_pb)$sampletype')
robjects.r('design <- model.matrix(~ 0 + group + replicate)')  
robjects.r('design <- subset(design, select = -c(replicateDEV16136_Quad, replicateDEV16569_Quad))')
#robjects.r('print(colnames(design))')
robjects.r('y <- estimateDisp(y, design = design)')
robjects.r('fit <- glmQLFit(y, design)')
robjects.r('fit <- glmQLFit(y, design)')
robjects.r('outs <- list("fit"=fit, "design"=design, "y"=y)')

In [ ]:
%%R 
y$samples

In [ ]:
%%R
plotMDS(y, col=ifelse(y$samples$group == "20w", "red", "blue"))

In [ ]:
%%R
plotBCV(y)

In [ ]:
%%R

colnames(y$design)

In [ ]:
%%R
# Search for positions of the three group names
positions <- grep(pattern = "group(12|17|20)w\\.White_*", x = colnames(y$design))

# Print the positions
print(positions)

In [ ]:
%%R
# Create a vector with the new names for the three variables
new_names <- c("group12w.White_Blood_Cells", "group17w.White_Blood_Cells", "group20w.White_Blood_Cells")

# Rename the variables in y$design
colnames(y$design)[c(13, 26, 38)] <- new_names

In [ ]:
adata_pb.obs['C_scANVI'] = adata_pb.obs['C_scANVI'].replace('White_‘Blood_Cells', 'White_Blood_Cells')

# check if the replacement is successful
print(adata_pb.obs['C_scANVI'].unique())

In [ ]:
%%R
# Get unique cell types
cell_types <- unique(colData(adata_pb)$C_scANVI)

# Replace 'White_‘Blood_Cells' with 'White_Blood_Cells'
cell_types <- gsub("White_‘Blood_Cells", "White_Blood_Cells", cell_types)

# Print the updated cell types
print(cell_types)

In [ ]:
robjects.r('de_per_cell_type_20vs12 <- list()')
robjects.r("""
for (cell_type in cell_types[c(1:3,5:13)]) {
    print(cell_type)
    # create contrast for this cell type
    myContrast <- makeContrasts(paste0("group20w.", cell_type, "-group12w.", cell_type), levels = y$design)
    # perform QLF test
    qlf <- glmQLFTest(fit, contrast=myContrast)
    # get all of the DE genes and calculate Benjamini-Hochberg adjusted FDR
    tt <- topTags(qlf, n = Inf)
    # save in the list with the results for all the cell types
    de_per_cell_type_20vs12[[cell_type]] <- tt$table
    }
""")
de_per_cell_type_20vs12 = robjects.r('de_per_cell_type_20vs12')

In [ ]:
robjects.r('de_per_cell_type_20vs17 <- list()')
robjects.r("""
for (cell_type in cell_types[c(1:3,5:13)]) {      #dropped Chondrocytes as they are not present in 20w samples
    print(cell_type)
    # create contrast for this cell type
    myContrast <- makeContrasts(paste0("group20w.", cell_type, "-group17w.", cell_type), levels = y$design)
    # perform QLF test
    qlf <- glmQLFTest(fit, contrast=myContrast)
    # get all of the DE genes and calculate Benjamini-Hochberg adjusted FDR
    tt <- topTags(qlf, n = Inf)
    # save in the list with the results for all the cell types
    de_per_cell_type_20vs17[[cell_type]] <- tt$table
    }
""")

de_per_cell_type20vs17 = robjects.r('de_per_cell_type_20vs17')

In [ ]:
cell_types = robjects.r('cell_types')
cell_types

In [ ]:
for i, cell_type in enumerate(cell_types):
    print(i, cell_type)
    

In [ ]:
ddf = robjects.conversion.rpy2py(de_per_cell_type)
ddf

In [ ]:
# add the table to .uns for each cell type
for i, cell_type in enumerate(cell_types):
    df = robjects.conversion.rpy2py(de_per_cell_type[i])
    df["gene_symbol"] = df.index
    df["cell_type"] = cell_type
    sc_toolbox.tools.de_res_to_anndata(
        adata,
        df,
        groupby="cell_type",
        score_col="logCPM",
        pval_col="PValue",
        pval_adj_col="FDR",
        lfc_col="logFC",
        key_added="edgeR_" + cell_type,
    )
    df.to_csv(os.path.join(RESULTS_FOLDERNAME, f"de_edgeR_17wVs12w_{cell_type}.csv"))

In [ ]:
ct = robjects.r('ct <- cell_types[c(1:3,5:13)]')
ct

In [ ]:
# add the table to .uns for each cell type
for i, cell_type in enumerate(ct):
    df = robjects.conversion.rpy2py(de_per_cell_type20vs17[i])
    df["gene_symbol"] = df.index
    df["cell_type"] = cell_type
    sc_toolbox.tools.de_res_to_anndata(
        adata,
        df,
        groupby="cell_type",
        score_col="logCPM",
        pval_col="PValue",
        pval_adj_col="FDR",
        lfc_col="logFC",
        key_added="edgeR_20vs17_" + cell_type,
    )
    df.to_csv(os.path.join(RESULTS_FOLDERNAME, f"de_edgeR_20wVs17w_{cell_type}.csv"))

In [ ]:
# add the table to .uns for each cell type
for i, cell_type in enumerate(ct):
    df = robjects.conversion.rpy2py(de_per_cell_type_20vs12[i])
    df["gene_symbol"] = df.index
    df["cell_type"] = cell_type
    sc_toolbox.tools.de_res_to_anndata(
        adata,
        df,
        groupby="cell_type",
        score_col="logCPM",
        pval_col="PValue",
        pval_adj_col="FDR",
        lfc_col="logFC",
        key_added="edgeR_20vs12_" + cell_type,
    )
    df.to_csv(os.path.join(RESULTS_FOLDERNAME, f"de_edgeR_20wVs12w_{cell_type}.csv"))

In [ ]:
adata

In [ ]:
adata.write(os.path.join(RESULTS_FOLDERNAME, 'dev_scANVI_DGE.h5ad'))

In [ ]:
sc.get.rank_genes_groups_df(adata, group="ABI3BP_Fibroblasts", key="edgeR_20vs12_ABI3BP_Fibroblasts")

In [ ]:
FDR = 0
LOG_FOLD_CHANGE = 0


def plot_heatmap_17vs12(adata, group_key, group_name="C_scANVI", groupby="age"):
    cell_type = "_".join(group_key.split("_")[1:])
    res = sc.get.rank_genes_groups_df(adata, group=cell_type, key=group_key)
    res.index = res["names"].values
    res = res[
        (res["pvals_adj"] < FDR) & (abs(res["logfoldchanges"]) > LOG_FOLD_CHANGE)
    ].sort_values(by=["logfoldchanges"])
    print(f"Plotting {len(res)} genes...")
    markers = list(res.index)
    sc.pl.heatmap(
        adata[adata.obs[group_name] == cell_type].copy(),
        markers,
        groupby=groupby,
        swap_axes=True,
    )

In [ ]:
FDR = 0.01
LOG_FOLD_CHANGE = 1.5


def plot_heatmap_20vs12(adata, group_key, group_name="C_scANVI", groupby="age"):
    cell_type = "_".join(group_key.split("_")[2:])
    res = sc.get.rank_genes_groups_df(adata, group=cell_type, key=group_key)
    res.index = res["names"].values
    res = res[
        (res["pvals_adj"] < FDR) & (abs(res["logfoldchanges"]) > LOG_FOLD_CHANGE)
    ].sort_values(by=["logfoldchanges"])
    print(f"Plotting {len(res)} genes...")
    markers = list(res.index)
    sc.pl.heatmap(
        adata[adata.obs[group_name] == cell_type].copy(),
        markers,
        groupby=groupby,
        swap_axes=True,
    )

In [ ]:
cell_type = "_".join('edgeR_20vs12_ABI3BP_Fibroblasts'.split("_")[2:])
cell_type

In [ ]:
res = sc.get.rank_genes_groups_df(adata, group=cell_type, key='edgeR_20vs12_ABI3BP_Fibroblasts')
res.index = res["names"].values
res = res[(res["pvals_adj"] < 0.01) & (abs(res["logfoldchanges"]) > 1.5)].sort_values(by=["logfoldchanges"])
res

In [ ]:
res

In [ ]:
print(f"Plotting {len(res)} genes...")
markers = list(res.index)
sc.pl.heatmap(
    adata[adata.obs["C_scANVI"] == cell_type].copy(),
    markers,
    groupby='age',
    swap_axes=True,
)

In [ ]:
plot_heatmap_20vs12(adata, "edgeR_20vs12_White_Blood_Cells")